In [1]:
import jieba
import jieba.posseg as pseg
import os
import xlrd
from openpyxl import Workbook
from opencc import OpenCC

In [2]:
#增設另外的詞庫
jieba.load_userdict("C:/Users/pro/jieba/鞋類擴充辭庫.txt")

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\pro\AppData\Local\Temp\jieba.cache
Loading model cost 1.103 seconds.
Prefix dict has been built succesfully.


In [3]:
#停用詞
symbol = "'" + '˚`°™"‘’”“®*^=+＋※◆★☆＊▲▼€¥￥╳.↑→-–—─、‧，·・~～⋯…,.【】《》〈〉<>/\[]{}()（）｢「」『』／$|！!？?%％#＃&@＠。：:；;\n'
stop_words = []
path_stop_words = open("C:/Users/pro/jieba/stop_words.txt", encoding = "utf-8-sig")
stop_words = path_stop_words.read().split("\n")

In [4]:
#品牌中文名稱轉英文名稱_function
brand = []

wb = xlrd.open_workbook("C:/Users/pro/Desktop/運動/品牌名稱中英對照.xlsx")
sheet = wb.sheet_by_index(0)
for i in range(1, sheet.nrows):
    nrow_eng = str(sheet.cell_value(i, 0)).lower().replace(" ", "_")
    nrow_chi = sheet.cell_value(i, 1).lower()
    if nrow_chi != "":
        for j in nrow_chi.split("、"):
            brand.append([nrow_eng, j, i])
            
def brand_name(content, brand = brand):
    for i in brand:
        if content.count(i[1]) > 0:
            content = [i[0] if x == i[1] else x for x in content]
    return content

In [5]:
#文章品牌標註_function
brand_shoes = []

wb = xlrd.open_workbook("C:/Users/pro/Desktop/運動/品牌名稱中英對照.xlsx")
sheet = wb.sheet_by_index(0)
for i in range(1, sheet.nrows):
    nrow_eng = str(sheet.cell_value(i, 0)).lower().replace(" ", "_")
    brand_shoes.append(nrow_eng)

# def brand_label(path_txt, brand_shoes = brand_shoes):
#     label = []
    
#     f = open(path_txt, encoding = "utf-8-sig")
#     content = f.read()
#     f.close()
    
#     for i in brand_shoes:
#         if i in content:
#             label.append(i)
#         elif i in path_txt.lower().replace(" ", "_").replace("-", "_"):
#             label.append(i)
            
#     return " ".join(label)

def brand_label(content_noun, brand_shoes = brand_shoes):
    label = []
    
    for i in brand_shoes:
        if i in content_noun:
            label.append(i)
            
    return " ".join(label)

In [6]:
#判斷極性(正負向)_function
positive=[]
negative=[]

for i in open("C:/Users/pro/jieba/positive_shoes.txt", encoding = "utf-8-sig"):
    positive.append(str(i))
for i in open("C:/Users/pro/jieba/negative_shoes.txt", encoding = "utf-8-sig"):
    negative.append(str(i))

def findpolarity(term):
    point=0
    for a in positive:
        if term.strip() == a.strip():
            point=1
    
    for b in negative:
        if term.strip() == b.strip():
            point=-1
    return point

In [10]:
#文章資料庫更新(評論網資料庫更新1206.xlsx)
wb = Workbook()
ws = wb.active
ws.append(["文章名稱", "日期", "網址", "網站", "原始文章", "所有詞性文章", "名詞文章", "情緒分數", "品牌", "分群"])

path_content = ["C:/Users/pro/Desktop/PTTshoes/", "C:/Users/pro/Desktop/PTTfemale-shoes/"]
path_url = ["C:/Users/pro/Desktop/url_PTTshoes.txt", "C:/Users/pro/Desktop/url_PTTfemale-shoes.txt"]

for j in range(2):
    urls = open(path_url[j], encoding = "utf-8-sig").read().split("\n")
    for file in os.listdir(path_content[j]):
        content_raw = open(path_content[j] + file, encoding = "utf-8-sig").read().replace("\n", "")
        if content_raw != "":
            content_all = []
            content_noun = []

            cc = OpenCC('s2t')
            content_raw = cc.convert(content_raw)
            content = content_raw.lower()
            content = content.replace("\x06", "").replace("\x08", "").replace("\x18", "").replace("\x1d", "").replace("\x1c", "")
            content = content.replace(" ", "_").replace("\xa0", "_").replace("\x7f", "_").replace("\t", "_")
            content = content.replace("ü", "u").replace("ç", "c").replace("č", "c").replace("è", "e").replace("š", "s")
            for i in symbol:
                if i in content:
                    content = content.replace(i, "_")

            for i in pseg.cut(content):
                if i.word.count("_") != len(i.word) and not i.word.isdecimal() and i.word not in stop_words:
                    content_all.append(i.word)
                    if i.flag[0].lower() == "n":
                        content_noun.append(i.word)

            brand = brand_label(content_noun)
            
            if brand != "":
                title = file[9:-4]
                date = file[:8]
                url = urls[urls.index(file)+1]
                if j == 0:
                    website = "PTTshoes"
                else:
                    website = "PTTfemale-shoes"
                    
                content_all = " ".join(brand_name(content_all))
                content_noun = " ".join(brand_name(content_noun))
                
                fraction = 0
                for i in content_all:
                    fraction += findpolarity(i)

                ws.append([title, date, url, website, content_raw, content_all, content_noun, fraction, brand])
                
            else:
                os.remove(path_content[j] + file)

        else:
            os.remove(path_content[j] + file)
    
wb.save("C:/Users/pro/Desktop/評論網資料庫更新1206.xlsx")

In [7]:
#文章資料庫更新(網站資料庫更新1206.xlsx)
file = ["C:/Users/pro/Desktop/更新1206.xlsx"]
wb_w = Workbook()
ws = wb_w.active
ws.append(["文章名稱", "日期", "網址", "網站", "原始文章", "所有詞性文章", "名詞文章", "情緒分數", "品牌", "分群"])

for f in file:
    wb = xlrd.open_workbook(f)
    sheet = wb.sheet_by_index(0)
    for i in range(sheet.nrows):
        title = sheet.cell_value(i, 1)
        date = sheet.cell_value(i, 0)
        url = sheet.cell_value(i, 3)
        website = sheet.cell_value(i, 4)
        content_raw = sheet.cell_value(i, 2).replace("\n", "")
        content_all = []
        content_noun = []
        
        cc = OpenCC('s2t')
        content_raw = cc.convert(content_raw)
        content = content_raw.lower()
        content = content.replace("\x06", "").replace("\x08", "").replace("\x18", "").replace("\x1d", "").replace("\x1c", "")
        content = content.replace(" ", "_").replace("\xa0", "_").replace("\x7f", "_").replace("\t", "_")
        content = content.replace("ü", "u").replace("ç", "c").replace("č", "c").replace("è", "e").replace("š", "s")
        for i in symbol:
            if i in content:
                content = content.replace(i, "_")

        for i in pseg.cut(content):
            if i.word.count("_") != len(i.word) and not i.word.isdecimal() and i.word not in stop_words:
                content_all.append(i.word)
                if i.flag[0].lower() == "n":
                    content_noun.append(i.word)
        
        fraction = 0
        for i in content_all:
            fraction += findpolarity(i)
        
        brand = brand_label(content_noun)
        content_all = " ".join(brand_name(content_all))
        content_noun = " ".join(brand_name(content_noun))
        
        if brand != "":
            ws.append([title, date, url, website, content_raw, content_all, content_noun, fraction, brand])
            
wb_w.save("C:/Users/pro/Desktop/網站資料庫更新1206.xlsx")

In [11]:
#資料庫更新1206.xlsx丟給lda判斷分群
f_title = open("C:/Users/pro/Desktop/標題.txt", "w", encoding = "utf-8")
f_content_noun = open("C:/Users/pro/Desktop/內文.txt", "w", encoding = "utf-8")
title = []
content_noun = []

wb = xlrd.open_workbook("C:/Users/pro/Desktop/資料庫更新1206.xlsx")
sheet = wb.sheet_by_index(0)
for i in range(1, sheet.nrows):
    title.append(sheet.cell_value(i, 0))
    content_noun.append(sheet.cell_value(i, 6))
    
f_title.write("\n".join(title))
f_content_noun.write("\n".join(content_noun))

f_title.close()
f_content_noun.close()

In [15]:
wb_w = Workbook()
ws = wb_w.active

wb = xlrd.open_workbook("C:/Users/pro/Desktop/LDA_theme.xls")
sheet = wb.sheet_by_index(0)
for i in range(0, sheet.nrows):
    ws.append([str(sheet.cell_value(i, 7).lstrip("[").rstrip("]").replace(",", ""))])
    
wb_w.save("C:/Users/pro/Desktop/LDA1206.xlsx")

In [165]:
# #找出所有文章中的英文片語
# phrase_eng = []

# path_article_old = "C:/Users/pro/Desktop/未斷詞文章/"
# for file in os.listdir(path_article_old):
#     article_new = ""

#     f = open(path_article_old + file, encoding = "utf-8-sig")
#     article = f.read().lower()
#     f.close()

#     #將文章中的符號替換成"_"
#     for i in symbol:
#         if i in article:
#             article = article.replace(i, "_")
            
#     article = article.replace("\x06", "").replace("\x08", "").replace("\x18", "").replace("\t", "").replace("\x1d", "").replace("\x1c", "").replace("\x7f", "")

#     #將文章中的中文替換成"/"
#     #將文章中的" "替換成"_"
#     for i in article:
#         if len(i) == len(i.encode("utf-8")):
#             article_new += i
#         else:
#             article_new += "/"
#     article_new = article_new.replace(" ", "_")

#     #找出文章中有的英文片語(list沒有重複的string)
#     #移除完全是數字的詞
#     for i in article_new.split("/"):
#         if len(i) != 0 and i != "_":
#             if not i.strip("_").replace("_", "").isdecimal():
#                 if len(i.strip("_").split()) <= 10: #片語的詞最多10個
#                     phrase_eng.append(i.strip().strip("_") + " n")
#     phrase_eng = list(dict.fromkeys(phrase_eng))

# phrase_eng.sort()
# print(phrase_eng)